### Import all necesaary packages


In [2]:
from requests import session
from bs4 import BeautifulSoup as bs
import re
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from urllib.parse import urlparse
from requests.exceptions import HTTPError
import mysql.connector


### Find all API categories


In [3]:
API_URL = 'https://rapidapi.com/categories'
with session() as s:
    r1 = s.get(API_URL)

soup = bs(r1.text, 'html.parser')
htmlText = soup.prettify()
API_Categories= soup.find_all('div',{'CollectionCarousel CollectionCarousel'})

### Get all API category links

In [4]:
# Parse the HTML content
soup_all_categories = bs(r1.text, 'html.parser')

# Find the <a> tag with the specified class
a_tags = soup.find_all('a', class_='CardLink')
href_values = []
categories = []
if a_tags:
    for a_tag in a_tags:
        href_value = a_tag.get('href')
        href_values.append(href_value)
        category = a_tag.text.strip()
        categories.append(category)      
else:
    print('No match found.')

No match found.


In [ ]:
if href_values:
    href_value = ["https://rapidapi.com" + href for href in href_values]    
    api_href_pairs = list(zip(categories, href_value))
    print(api_href_pairs)
else:
    print('No href value found.')

### Get API links for each category link


In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run Chrome in headless mode, without opening a window

user_data_dir = r'C:\xxx\AppData\Local\Google\Chrome\User Data'

webdriver_path = 'D:\chromedriver_win32\chromedriver.exe'  # Replace with the actual path to the chromedriver executable

options.add_argument(f'--user-data-dir={user_data_dir}')

driver = webdriver.Chrome(executable_path=webdriver_path, options=options)

#for x in href_value:
    #print(x)
    # Load the initial URL
driver.get('https://rapidapi.com/category/News,%20Media')
base_urls = []
def scrape_page(url):
    soup = bs(driver.page_source, 'html.parser')
    a_tags = soup.find_all('a', class_='CardLink')
    if a_tags:
        for a_tag in a_tags:
            base_url = "https://rapidapi.com" + a_tag.get('href')
            base_urls.append(base_url)
def scrape_all_pages(base_url):
     while True:
        scrape_page(driver.current_url)
        next_button = None
        try:
            next_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(@class, 'r-page-link') and contains(text(),'›') and not(@disabled)]")))
            driver.execute_script("arguments[0].click();", next_button)
            WebDriverWait(driver, 10).until(EC.staleness_of(next_button))
        except:
            if next_button is None:
                break

scrape_all_pages('https://rapidapi.com/category/News,%20Media')
driver.quit()  # Close the browser after scraping    

# Print the scraped URLs
for url in base_urls:
    print(url)


### Get Details from Pricing Page 


In [ ]:
pricing_details = {}

for base_url in base_urls:
    print(base_url)
    try:
        r4 = s.get(base_url + "pricing")
    except TimeoutException:
        pricing = 'No pricing details available'
        print(pricing)
    soup4 = bs(r4.text, 'html.parser')
    plan_names = []
    plan_requests = []
    plan_costs = []

    plan_name_elements = soup4.find_all("h2", class_="PlanName")
    plan_request_elements = soup4.find_all("div", class_="LimitQuotaRow bold")
    plan_cost_per_req_elements = soup4.find_all("div", class_="LimitOverageRow caption")
#     td_element = soup4.find('td', class_='PlanDetailCell').text.strip()
#     if "requests per" in td_element:
#         plan_request_elements = re.search(r'\b(\d+)\b', td_element)
#         if plan_request_elements:
#             plan_request_elements = plan_request_elements.group(1)

#     print(plan_request_elements)   

    
    # Initialize variables
    basic = None
    pro = None
    ultra = None
    mega = None
    basic_cost_per_req = None
    pro_cost_per_req = None
    ultra_cost_per_req = None
    mega_cost_per_req = None
    hard_limit = None

    for plan_name_element, plan_request_element, plan_cost_per_req_element in zip(plan_name_elements, plan_request_elements, plan_cost_per_req_elements):
        plan_name = plan_name_element.text.strip()
        plan_cost_per_req = plan_cost_per_req_element.text.strip()
        plan_request = plan_request_element.text.strip()
        print(plan_request)
        plan_names.append(plan_name)
        plan_requests.append(plan_request)
        parts = plan_cost_per_req.split('$')
            
        if len(parts) == 2:
            print(parts)
            plan_cost_per_req = float(parts[1].split()[0].replace(',', ''))
            
            plan_costs.append(plan_cost_per_req)               

        else:
            plan_cost_per_req = 0
            plan_costs.append(plan_cost_per_req)
                
    if len(plan_costs) == 0 and len(plan_requests) == 0:
        pricing_details[base_url] = {
            'basic_req': basic,
            'pro_req': pro,
            'ultra_req': ultra,
            'mega_req': mega,
            'basic_per_req_cost': basic_cost_per_req,
            'pro_per_req_cost': pro_cost_per_req,
            'ultra_per_req_cost': ultra_cost_per_req,
            'mega_per_req_cost': mega_cost_per_req
        }
        
    # Iterate over plan_names to set variables
    for name, request, cost in zip(plan_names, plan_requests, plan_costs):
        if "Basic" in name:
            basic_cost_per_req = cost
            if ',' in request:
                request = request.replace(',', '')
            basic = int(request.split('/')[0].strip())
            if "month" in request:
                basic = basic/30
        elif "Pro" in name:
            pro_cost_per_req = cost
            if ',' in request:
                request = request.replace(',', '')
            pro = int(request.split('/')[0].strip())
            if "month" in request:
                pro = pro/30
        elif "Ultra" in name:
            ultra_cost_per_req = cost
            if ',' in request:
                request = request.replace(',', '')
            ultra = int(request.split('/')[0].strip())
            if "month" in request:
                ultra = ultra/30
        elif "Mega" in name:
            mega_cost_per_req = cost
            if ',' in request:
                request = request.replace(',', '')
            mega = int(request.split('/')[0].strip())
            if "month" in request:
                mega = mega/30

        # Store pricing details for the current base_url
        pricing_details[base_url] = {
            'basic_req': basic,
            'pro_req': pro,
            'ultra_req': ultra,
            'mega_req': mega,
            'basic_per_req_cost': basic_cost_per_req,
            'pro_per_req_cost': pro_cost_per_req,
            'ultra_per_req_cost': ultra_cost_per_req,
            'mega_per_req_cost': mega_cost_per_req
        }
    


### Upload to Table api_pricing

In [ ]:
import mysql.connector

# Connect to MySQL
cnx = mysql.connector.connect(
    host='xxx',
    user='xxx',
    password='xxx',
    database='rapAPI2'
)

cursor = cnx.cursor()

# Prepare the SQL statement for the insert operation
insert_endpoint_details = "INSERT INTO api_pricing (base_url, basic_req, pro_req, ultra_req, mega_req, " \
                          "basic_per_req_cost, pro_per_req_cost, ultra_per_req_cost, mega_per_req_cost) " \
                          "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"

# Iterate over the items in the api_details dictionary
for base_url, details in pricing_details.items():
    base_url = base_url[:255]
    basic_req = details['basic_req']
    pro_req = details['pro_req']
    ultra_req = details['ultra_req']
    mega_req =  details['mega_req']
    basic_per_req_cost = details['basic_per_req_cost']
    pro_per_req_cost = details['pro_per_req_cost']
    ultra_per_req_cost = details['ultra_per_req_cost']
    mega_per_req_cost = details['mega_per_req_cost']

    # Check if URL already exists in the database
    select_query = "SELECT base_url FROM api_pricing WHERE base_url = %s"
    cursor.execute(select_query, (base_url,))
    existing_url = cursor.fetchone()

    if existing_url:
        print(f"URL {base_url} already exists in the database. Skipping insertion.")
    else:
        # Execute the insert operation for each URL and details
        cursor.execute(insert_endpoint_details, (base_url, basic_req, pro_req, ultra_req, mega_req, basic_per_req_cost,
                                                pro_per_req_cost, ultra_per_req_cost, mega_per_req_cost))

# Commit the changes and close the connection
cnx.commit()
cnx.close()
